## Data upload

In [1]:
#import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random
# Device configuration
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [2]:
#update data
#feature
train_ch = np.array(pd.read_csv("train_noise_logits_before.csv", index_col = 0))
#origin input
train = np.array(pd.read_csv("train.csv", index_col = 0))
train = np.reshape(train, [train.shape[0], 2, 128])
#label
train_label = np.array(pd.read_csv("train_label.csv", index_col = 0))
train_noise_label = np.array(pd.read_csv("train_noise_label.csv", index_col = 0))
train_label = np.reshape(train_label, [-1])
train_noise_label = np.reshape(train_noise_label, [-1])

In [3]:
sum(train_noise_label == train_label) / len(train_noise_label)

0.5

In [4]:
a = np.cov(train_ch, rowvar = False)
a_c, a_v = np.linalg.eig(a)
train_ch_ro = np.dot(train_ch, a_v)

## CSFE

In [5]:
def g(x, sample, label, r):
    n  = sample.shape[0]
    g = 0

    for i in range(r):
        p_r = np.sum(label == i) / n
        F_n = np.sum((sample <= x)) / n
        F_n_bar = 1 - F_n
        F_rn = np.sum((sample[(label == i)] <= x)) / (n * p_r)
        F_rn_bar = 1 - F_rn
#         print(F_n, F_rn)
        
        g += p_r * F_rn * np.log((F_rn + 1e-6) / (F_n + 1e-6)) + p_r * F_rn_bar * np.log((F_rn_bar + 1e-6) / (F_n_bar + 1e-6))
        
        
#         if (F_n != 0) & (F_n_bar != 0) & (F_rn != 0) & (F_rn_bar != 0):
#             g += p_r * F_rn * np.log(F_rn / (F_n)) + p_r * F_rn_bar * np.log(F_rn_bar / (F_n_bar))
#         else:
#             g += 0
#     print(g)

    return 2 * g   

In [6]:
def LR(x, y, r):
    n = x.shape[0]
    p = x.shape[1]
    lr = np.zeros(p)
    for i in range(p):
        sample = x[:, i]
        label = y
        m = np.min(sample)
        M = np.max(sample)
        x_i = np.linspace(m, M, 101)
        inter = (M - m) / 100
        j = 0
        for j in range(len(x_i) - 1):
            lr[i] += g(x_i[j], sample, label, r) / n
    
    
    return lr

In [7]:
lr_result = LR(train_ch_ro, train_noise_label, 11)

In [8]:
np.set_printoptions(threshold=100000)
lr_result

array([1.10721441e-03, 1.23642569e-03, 4.10178095e-04, 6.72735976e-04,
       2.60972664e-04, 6.52503019e-05, 1.62455994e-04, 1.51640336e-04,
       1.22306409e-04, 1.13329201e-04, 9.26008110e-05, 6.81021669e-05,
       1.10447851e-04, 1.67670596e-04, 1.01777107e-04, 6.59312249e-05,
       8.77701079e-05, 8.78169816e-05, 9.39992184e-05, 6.72920109e-05,
       7.38196066e-05, 8.62993488e-05, 6.58774616e-05, 7.81064847e-05,
       6.43273277e-05, 9.69365057e-05, 7.04574735e-05, 6.81592562e-05,
       6.06637836e-05, 6.11144843e-05, 5.19339644e-05, 7.61021226e-05,
       6.04431267e-05, 8.65615143e-05, 6.61927828e-05, 9.77094734e-05,
       6.65392146e-05, 5.52382118e-05, 5.90695636e-05, 5.68459348e-05,
       7.16315234e-05, 8.00030165e-05, 6.68455219e-05, 5.86260510e-05,
       6.74720106e-05, 7.23778566e-05, 5.63365991e-05, 6.86554525e-05,
       6.14824820e-05, 4.88273151e-05, 6.32969182e-05, 6.95132125e-05,
       6.03801170e-05, 6.80873380e-05, 4.77802093e-05, 7.36358329e-05,
      

In [9]:
np.argsort(lr_result)

array([137, 136, 172, 181, 139, 179, 192, 135, 206,  77, 208, 152, 221,
        57, 204, 242, 244, 201, 205,  91, 183, 211, 233, 104, 154, 229,
        69, 235, 200,  72,  54, 198, 163,  79, 164,  49, 252,  84,  65,
       215, 153, 226,  82, 223, 196, 110, 251, 212, 228, 160, 187,  62,
       178, 234, 158, 202,  78, 214, 218, 210,  30,  59,  88, 186,  66,
       255, 225, 249,  76, 197, 134, 127, 239, 175,  95,  56,  37, 111,
       157,  73, 185, 191, 130, 156, 120,  70, 207,  87,  46, 173,  63,
       224, 253, 108,  39, 182, 115, 161, 103, 114, 155, 133, 241, 193,
       147,  68,  43, 119, 116, 195,  38, 169, 230,  89, 101, 236,  74,
       141, 150, 131,  97, 232,  98,  52,  32,  90, 220,  28, 149,  29,
       117, 165,  96, 238, 146, 254,  48, 100, 247, 113,  86, 166, 222,
        83, 143, 159, 138,  50, 144, 217,  85, 132, 180, 194, 213,  24,
        93, 142, 107, 189,   5, 148,  99, 209, 199, 105, 219,  22, 118,
        15, 102,  64, 171,  34, 246, 151, 145, 240,  36, 162,  4

## TGC

In [10]:
def t_type(x, v):
    n = x.shape[0]
    p = x.shape[1]
    cov = np.identity(p)
    mu = np.zeros([1, p])
    mmu = np.zeros([n, p])
    a = 1 - (p < n)
    iter_num = 50
    err = 10
    for i in range(iter_num):
        if err <= 1e-12:
            break
        else:
            oldcov = cov
            incov = np.linalg.inv(cov)
            temp = np.dot(np.dot((x - mmu), incov), (x - mmu).T)
            wt = (v + p) / (v + np.diag(temp))
#             print(wt.shape)
            mu = np.dot(wt, x) / np.sum(wt)
            mmu = np.dot(np.ones([n, 1]), np.reshape(mu, [1, -1]))
#             print(mmu.shape)
            diag_wt = np.diag(wt)
#             print(diag_wt.shape)
            cov1 = np.dot(np.dot((x - mmu).T, diag_wt), (x - mmu)) / n
            cov = cov1 / (1 + a) + a * np.identity(p) / (1 + a)
            cov = cov
            temp1 = (p * oldcov / np.sum(np.diag(oldcov))) - (p * cov / np.sum(np.diag(cov)))
            err = np.linalg.norm(temp1, ord = "fro")
#             print(i)
    
    
    return mu, cov

## Training Set Boundaries Reassigned

In [135]:
def get_dis(x, mu, cov, inv, det):
    n = x.shape[0]
    up = 2000
    tem = x - mu
    tem = np.reshape(tem, [1, -1])
    dis0 = np.dot(np.dot(tem, inv), tem.T)
    dis = dis0 + 10 * det
    return dis 

In [136]:
def pre(x, y, v):
    a = 700
    up = 2000
    n = x.shape[0]
    p = x.shape[1]
    mu = np.zeros(p)
    cov = np.zeros([p, p])
    inv = np.zeros([p, p])
    dis_all0 = np.zeros([n, 11])
    dis_all = np.zeros([n, 11])
    for i in range(11):
        temp0 = x[(y == i), :]
        mu, cov = t_type(temp0, v)
        inv = np.linalg.inv(cov)
        eig, vector = np.linalg.eig(cov)
        ln_det = np.sum(np.log(eig))
        m = 0
        for m in range(n):
            tem1 = x[m, :]
            dis_all[m, i] = get_dis(tem1, mu, cov, inv, ln_det)
    return dis_all

In [137]:
def acc(x, y, v, yt):
    d = pre(x, y, v)
    n = d.shape[0]
    pre_label = np.zeros(n)
    correct = 0
    for i in range(n):
        pre_label[i] = np.argsort(d[i, :])[0]
        if pre_label[i] == yt[i]:
            correct += 1
    return correct / n, pre_label

In [309]:
train_temp = train_ch_ro[:, np.argsort(lr_result)[1:]]
acc_rate, pre_label = acc(train_temp, train_noise_label, 1, train_label)
acc_rate

0.5872727272727273

In [310]:
train_temp = train_ch_ro[:, np.argsort(lr_result)[1:]]
acc_rate, pre_label = acc(train_temp, train_noise_label, 2, train_label)
acc_rate

0.5885714285714285

In [314]:
train_temp = train_ch_ro[:, np.argsort(lr_result)[150:]]
acc_rate, pre_label = acc(train_temp, train_noise_label, 3, train_label)
acc_rate

0.6874025974025973

In [312]:
train_temp = train_ch_ro[:, np.argsort(lr_result)[1:]]
acc_rate, pre_label = acc(train_temp, train_noise_label, 4, train_label)
acc_rate

0.5902597402597403

In [313]:
train_temp = train_ch_ro[:, np.argsort(lr_result)[1:]]
acc_rate, pre_label = acc(train_temp, train_noise_label, 5, train_label)
acc_rate

0.5941558441558441

In [315]:
pd.DataFrame(pre_label).to_csv("train_reassigned_label_LR_after150.csv")

In [316]:
acc_rate

0.6874025974025973